# 'with' statement
- very common pattern in software is to aquire some kind of resource or context,
use it for awhile, then return it or undo it. 
- common examples are file and network descriptors
    - very important to use descriptors correctly
    - running out of descriptors can crash a server

In [1]:
# tedious to write all this out

import tempfile

tmp = tempfile.NamedTemporaryFile().name

fd = open(tmp, 'w')
# do things that might fail somehow
try:
    fd.write('foo')
finally: 
    # error or not, want to close the file descriptor
    # finally clause guarantees close will happen
    fd.close()

In [2]:
# instead, use 'with'
# less work, more consise

with open(tmp, 'w') as fd:
    fd.write('foo')


# Example - dominate module
- surpising and elegant implementation of a HTML builder using 'with' context protocol
- must install it:
    - pip install dominate
- [doc](https://github.com/Knio/dominate)

In [3]:
# will use this in an example soon

import dominate
from dominate.tags import *

doc = dominate.document(title='HTML via with')

with doc.head:
    link(rel='stylesheet', href='style.css')
    script(type='text/javascript', src='script.js')

with doc.body:
    with div(id='header').add(ol()):
        for j in range(3):
            for t in ['mp3', 'pdf']:
                li(a(f'{j}.{t}', href=f'data/{j}.{t}'))


print(doc)

ModuleNotFoundError: No module named 'dominate'

In [4]:
# dominate can also use decorators...

@div
def greeting(name):
    p('Hello %s' % name)
print(greeting('Bob'))

NameError: name 'div' is not defined

# 'with' implements 'context manager' protocol
- like iteration protocol, a general protocol implemented by many classes
- ```__enter__``` method - called at start of with block
    - allocate resources
- do work with resources
- ```__exit__``` method - called at end of with block, or when error raised
    - release resources

In [5]:
class File():

    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode

    def __enter__(self):
        print('enter')
        self.open_file = open(self.filename, self.mode)
        # as variable bound to this
        return self.open_file

    def __exit__(self, *args):
        print('exit')
        self.open_file.close()

In [6]:
import tempfile
        
with File('/tmp/foo.txt', 'w') as fd:
    print('here')
    fd.write('foo')
    print('there')

enter
here
there
exit


In [7]:
# 'with' use above roughly equivalent to:

f = File(tmp, 'w')
fd = f.__enter__()
try:
    fd.write('foo')
finally:
    # always executed, closes the file descriptor
    f.__exit__()


enter
exit


#  use decorators
and generators to implement a context manager
- [doc](https://docs.python.org/3/library/contextlib.html)

# Make a context manager for the current working directory

In [8]:
from contextlib import contextmanager
import os

@contextmanager
def withChdir(newdir):
    savedir = os.getcwd()
    os.chdir(newdir)
    try:
        yield
    finally:
        os.chdir(savedir)

In [9]:
# current working dir
print(os.getcwd())

# change it inside with context
with withChdir('/'):
    print( os.getcwd())
        
# dir before with is restored
print(os.getcwd()) 

/Users/dbenson30/Desktop/python/lectures
/
/Users/dbenson30/Desktop/python/lectures
